In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# dx = 1 km; Np = 1M; Nt = 5 min
data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc', decode_timedelta=True) #***
parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc', decode_timedelta=True) #***
res='1km';t_res='5min'
Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc', decode_timedelta=True) #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 50M; Nz = 95
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc', decode_timedelta=True) #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min_95nz'; Np_str='50e6'

# # dx = 250km; Np = 50M 
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_250m_1min_50M.nc', decode_timedelta=True) #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_250m_1min_50M.nc', decode_timedelta=True) #***
# res='250m'; t_res='1min'; Np_str='50e6'

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
if res=='1km':
    dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
if res=='250m':
    dir2='/mnt/lustre/koa/scratch/air673/'
def initiate_array():
    # Define array dimensions (adjust based on your netCDF)
    t_size = len(data['time'])  # Number of timesteps
    z_size = len(data['zh'])    # Number of vertical levels
    y_size = len(data['yh'])    # Number of y-axis points
    x_size = len(data['xh'])    # Number of x-axis points

    out_file = dir2 + 'Variable_Calculation/OUTPUT/' + f'Eulerian_Binary_Array_{res}_{t_res}.h5'
    with h5py.File(out_file, 'a') as f:
        for var_name in ['A_c', 'A_g']:
            if var_name not in f:
                f.create_dataset(var_name, 
                                 (t_size, z_size, y_size, x_size), 
                                 maxshape=(None, z_size, y_size, x_size),
                                 dtype='bool', 
                                 chunks=(1, z_size, y_size, x_size))


            
def add_timestep_at_index(timestep_data_dict, index):
    out_file = dir2 + 'Variable_Calculation/OUTPUT/' + f'Eulerian_Binary_Array_{res}_{t_res}.h5'
    with h5py.File(out_file, 'a') as f:
        if 'A_c' in timestep_data_dict:
            f['A_c'][index] = timestep_data_dict['A_c']
        if 'A_g' in timestep_data_dict:
            f['A_g'][index] = timestep_data_dict['A_g']

In [ ]:
####################################
#RUNNING

In [ ]:
w_thresh1=0.1
w_thresh2=0.5
qcqi_thresh=1e-6

In [ ]:
initiate_array()

#CALCULATING AND APPENDING TO DATA EACH TIMESTEP
for t in range(len(data['time'])):
    if np.mod(t,1)==0: print(f'Current time {t}')

    print('calculating variables')
    w_data=data['winterp'].isel(time=t).data
    qc_data=data['qc'].isel(time=t).data
    qi_data=data['qi'].isel(time=t).data
    qcqi_data=qc_data+qi_data
        
    print('calculating A_c binary array')
    cond1 = w_data >= w_thresh2
    cond2 = qcqi_data >= qcqi_thresh
    A_c = np.where(cond1 & cond2, True, False)

    print('calculating A_g binary array')
    cond1 = w_data >= w_thresh1
    cond2 = qcqi_data < qcqi_thresh
    A_g = np.where(cond1 & cond2, True, False)

    print('outputting')
    result = {
        'A_c': A_c,
        'A_g': A_g
    }
    add_timestep_at_index(result, t)


In [ ]:
######################################################

In [ ]:
# #READING BACK IN
# dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
# out_file = dir2 + 'Variable_Calculation/' + f'Eulerian_Binary_Array_{res}_{t_res}.h5'

# with h5py.File(out_file, 'r') as f:
#     A_c = f['A_c'][t]  # Loads the entire A_c array into memory
#     A_g = f['A_g'][t]  # Loads the entire A_g array into memory

In [ ]:
# #####################
# #TESTING
# w_thresh1=0.1
# w_thresh2=0.5
# qcqi_thresh=1e-6
# z=10
# w_t=data['winterp'].isel(time=t,zh=z).data
# qc_t=data['qc'].isel(time=t,zh=z).data
# qi_t=data['qi'].isel(time=t,zh=z).data
# qcqi_t=qc_t+qi_t

# out_file = dir2 + 'Variable_Calculation/' + f'Eulerian_Binary_Array_{res}_{t_res}.h5'
# with h5py.File(out_file, 'r') as f:
#     A_c = f['A_c'][t,z]  # Loads the entire A_c array into memory
#     A_g = f['A_g'][t,z]  # Loads the entire A_g array into memory

# cond1=(w_t>=w_thresh1)
# cond2=(qcqi_t<qcqi_thresh)
# print(np.all((cond1&cond2)==A_g))

# cond1=(w_t>=w_thresh2)
# cond2=(qcqi_t>=qcqi_thresh)
# print(np.all((cond1&cond2)==A_c))